### **Integer programming**

Consider the following integer program:

$$\begin{align}\min_{x_1,x_2} \quad & x_2 \\
\mathrm{s.t.} \quad & 2 x_1 + x_2 \geq 13 \\
& 5 x_1 + 2 x_2 \leq 30 \\
& -x_1 + x_2 \geq 5 \\
& x_1, x_2 \in \mathcal{Z} := \{0,1,2,...\}
\end{align}$$

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()
solver = po.SolverFactory("glpk")

In [3]:
m.x1 = pe.Var(domain=pe.PositiveIntegers)
m.x2 = pe.Var(domain=pe.PositiveIntegers)

# Constraint 1
m.con1 = pe.Constraint(expr=2*m.x1 + m.x2 >= 13)

# Constraint 2
m.con2 = pe.Constraint(expr=5*m.x1 + 2*m.x2 <= 30)

# Constraint 3
m.con3 = pe.Constraint(expr=-m.x1 + m.x2 >= 5)

# Objective
m.obj = pe.Objective(expr=m.x2)

m.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     1 :  None :  None : False :  True : PositiveIntegers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     1 :  None :  None : False :  True : PositiveIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize :         x2

3 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  13.0 : 2*x1 + x2 :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :  -Inf : 5*x1 + 2*x2 :  30.0 :   True
    con3 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :   5.0 : - x1 + x2 :  +Inf :   True

6 Declarations: x1 x2 con1 con2 con3

In [4]:
solver.solve(m,tee=True)

# Print solution
print("x1 = ",m.x1())
print("x2 = ",m.x2())

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Lenovo\AppData\Local\Temp\tmp9pwvarip.glpk.raw --wglp C:\Users\Lenovo\AppData\Local\Temp\tmpfj0obp9x.glpk.glp
 --cpxlp C:\Users\Lenovo\AppData\Local\Temp\tmp93tkyr2n.pyomo.lp
Reading problem data from 'C:\Users\Lenovo\AppData\Local\Temp\tmp93tkyr2n.pyomo.lp'...
4 rows, 3 columns, 7 non-zeros
2 integer variables, none of which are binary
33 lines were read
Writing problem data to 'C:\Users\Lenovo\AppData\Local\Temp\tmpfj0obp9x.glpk.glp'...
25 lines were written
GLPK Integer Optimizer, v4.65
4 rows, 3 columns, 7 non-zeros
2 integer variables, none of which are binary
Preprocessing...
3 rows, 2 columns, 6 non-zeros
2 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+00  ratio =  5.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
3 rows, 

#### Knapsack optimization  --> Binary variables

$$\begin{align}\max_{x_i} \quad &  \sum^{i=1}_{n} v_ix_i \\
\mathrm{s.t.} \quad & \sum^{i=1}_{n} w_ix_i \leq W \\
& x_i \in \{0,1\}
\end{align}$$

In [5]:
ks = pe.ConcreteModel()
solver = po.SolverFactory("glpk")

ks.I = pe.RangeSet(4)
ks.x = pe.Var(ks.I, domain=pe.Binary)

v = {
    1: 11,
    2: 8,
    3: 3,
    4: 6}
w = {
    1: 3,
    2: 5,
    3: 7,
    4: 4}

ks.v = pe.Param(ks.I, initialize=v)
ks.w = pe.Param(ks.I, initialize=w)
ks.W = pe.Param(initialize=14.0)

# Constraint 
ks.con1 = pe.Constraint(expr=(
    sum(ks.x[i]*ks.w[i] for i in ks.I) <= ks.W
))

# Objective
ks.obj = pe.Objective(
    expr=sum(ks.x[i]*ks.v[i] for i in ks.I),
    sense=pe.maximize)

solver.solve(ks)

for i in ks.I:
    print(pe.value(ks.x[i]))

1.0
1.0
0.0
1.0
